Install AWS CLI [installed][l_cli] and [configure it][l_cli_config]. Consider using [named profiles][l_namedprofiles].

Import boto3 and AWS create clients.

[l_cli]: http://docs.aws.amazon.com/cli/latest/userguide/installing.html
[l_cli_config]: (http://docs.aws.amazon.com/cli/latest/userguide/cli-chap-getting-started.html)
[l_namedprofiles]: (https://docs.aws.amazon.com/cli/latest/userguide/cli-multiple-profiles.html)

In [ ]:
import json
import yaml
import boto3
gg = boto3.client('greengrass')
iot = boto3.client('iot')

In [ ]:
 group = gg.create_group(Name="my_group")

In [ ]:
keys_cert = iot.create_keys_and_certificate(setAsActive=True)

In [ ]:
core_thing = iot.create_thing(thingName="my_group_core_1")

In [ ]:
iot.attach_thing_principal(thingName=core_thing['thingName'], principal=keys_cert['certificateArn'])

In [ ]:
core_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "iot:Publish",
                "iot:Subscribe",
                "iot:Connect",
                "iot:Receive",
                "iot:GetThingShadow",
                "iot:DeleteThingShadow",
                "iot:UpdateThingShadow"
            ],
            "Resource": ["arn:aws:iot:" + boto3.session.Session().region_name + ":*:*"]
        },
        {
            "Effect": "Allow",
            "Action": [
                "greengrass:AssumeRoleForGroup",
                "greengrass:CreateCertificate",
                "greengrass:GetConnectivityInfo",
                "greengrass:GetDeployment",
                "greengrass:GetDeploymentArtifacts",
                "greengrass:UpdateConnectivityInfo",
                "greengrass:UpdateCoreDeploymentStatus"
            ],
            "Resource": ["*"]
        }
    ]
}

In [ ]:
policy = iot.create_policy(
    policyName="my_core_1_policy",
    policyDocument=json.dumps(core_policy_doc)
)

In [ ]:
iot.attach_principal_policy(
    policyName=policy['policyName'],
    principal=keys_cert['certificateArn']
)

In [ ]:
initial_version = {'Cores': []}
initial_version['Cores'].append({
    'Id': core_thing['thingName'], # Quite intuitive, eh?
    'CertificateArn': keys_cert['certificateArn'],
    'SyncShadow': False, # Up to you, True|False
    'ThingArn': core_thing['thingArn']
})

In [ ]:
core_definition = gg.create_core_definition(
    Name="{0}_core_def".format(group['Name']),
    InitialVersion=initial_version
)

There is nothing yet at AWS Console at this point. We just lined up the pieces.
Let's get to the real business.

In [ ]:
group_ver = gg.create_group_version(
    GroupId=group['Id'],
    CoreDefinitionVersionArn=core_definition['LatestVersionArn']
)

Now the group is manifested in AWS console.
Go check there. Or run the script below to get the links.

PS. I can't figure why the core created via API doesn't show up under Greengrass/Cores.

In [ ]:
region = boto3.session.Session().region_name
print ("Group: https://.console.aws.amazon.com/iot/home?region={}#/greengrass/groups/{}".format(region, group['Id']))
print ("Core thing: https://{0}.console.aws.amazon.com/iot/home?{0}#/thing/my_group_core_1".format(region))
print ("Certificate and policy: https://{0}.console.aws.amazon.com/iot/home?region={0}#/certificate/{1}".format(region, keys_cert['certificateArn']))

### Save the state
Save created entities to the file. 
You'll thank me for that when it's time to clean things up.

In [ ]:
state = {
    'group': group,
    'core_thing': core_thing,
    'keys_cert': keys_cert,
    'group_ver': group_ver,
    'core_definition': core_definition,
    'policy': policy
}

In [ ]:
with open('./state.json', 'w') as f:
    json.dump(state, f, indent=4)

In [ ]:
state